In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [45]:
df = pd.read_csv('solar_flare.csv')
df.head()

,flare,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5
0,2021213,2002-02-12,21:29:56,21:33:38,21:41:48,712,136,167304.0,12-25,592,-358,692,0,A1,P1,NaN,NaN,NaN
1,2021228,2002-02-12,21:44:08,21:45:06,21:48:56,288,7,9504.0,6-12,604,-341,694,9811,A1,P1,PE,Q1,NaN
2,2021332,2002-02-13,00:53:24,00:54:54,00:57:00,216,15,11448.0,6-12,-310,375,487,9825,A1,P1,NaN,NaN,NaN
3,2021308,2002-02-13,04:22:52,04:23:50,04:26:56,244,20,17400.0,12-25,-277,378,469,9822,A1,P1,NaN,NaN,NaN
4,2021310,2002-02-13,07:03:52,07:05:14,07:07:48,236,336,313392.0,25-50,-272,390,476,9825,A1,GS,P1,PE,Q2


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116143 entries, 0 to 116142
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   flare             116143 non-null  int64  
 1   start.date        116143 non-null  object 
 2   start.time        116143 non-null  object 
 3   peak              116143 non-null  object 
 4   end               116143 non-null  object 
 5   duration.s        116143 non-null  int64  
 6   peak.c/s          116143 non-null  int64  
 7   total.counts      116143 non-null  float64
 8   energy.kev        116143 non-null  object 
 9   x.pos.asec        116143 non-null  int64  
 10  y.pos.asec        116143 non-null  int64  
 11  radial            116143 non-null  int64  
 12  active.region.ar  116143 non-null  int64  
 13  flag.1            116143 non-null  object 
 14  flag.2            116143 non-null  object 
 15  flag.3            96236 non-null   object 
 16  flag.4            95

In [47]:
df.isnull().sum()

flare                   0
start.date              0
start.time              0
peak                    0
end                     0
duration.s              0
peak.c/s                0
total.counts            0
energy.kev              0
x.pos.asec              0
y.pos.asec              0
radial                  0
active.region.ar        0
flag.1                  0
flag.2                  0
flag.3              19907
flag.4              20202
flag.5              61180
dtype: int64

In [48]:
df = df.drop(['flare','flag.1','flag.2','flag.3', 'flag.4', 'flag.5'], axis=1)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116143 entries, 0 to 116142
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   start.date        116143 non-null  object 
 1   start.time        116143 non-null  object 
 2   peak              116143 non-null  object 
 3   end               116143 non-null  object 
 4   duration.s        116143 non-null  int64  
 5   peak.c/s          116143 non-null  int64  
 6   total.counts      116143 non-null  float64
 7   energy.kev        116143 non-null  object 
 8   x.pos.asec        116143 non-null  int64  
 9   y.pos.asec        116143 non-null  int64  
 10  radial            116143 non-null  int64  
 11  active.region.ar  116143 non-null  int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 10.6+ MB


In [50]:
df['start.time'] = pd.to_datetime(df['start.date'] + ' ' + df['start.time'])
df['peak'] = pd.to_datetime(df['start.date'] + ' ' + df['peak'])
df['end'] = pd.to_datetime(df['start.date'] + ' ' + df['end'])
df.head()

,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar
0,2002-02-12,2002-02-12 21:29:56,2002-02-12 21:33:38,2002-02-12 21:41:48,712,136,167304.0,12-25,592,-358,692,0
1,2002-02-12,2002-02-12 21:44:08,2002-02-12 21:45:06,2002-02-12 21:48:56,288,7,9504.0,6-12,604,-341,694,9811
2,2002-02-13,2002-02-13 00:53:24,2002-02-13 00:54:54,2002-02-13 00:57:00,216,15,11448.0,6-12,-310,375,487,9825
3,2002-02-13,2002-02-13 04:22:52,2002-02-13 04:23:50,2002-02-13 04:26:56,244,20,17400.0,12-25,-277,378,469,9822
4,2002-02-13,2002-02-13 07:03:52,2002-02-13 07:05:14,2002-02-13 07:07:48,236,336,313392.0,25-50,-272,390,476,9825


In [51]:
df = df.drop('start.date', axis=1)

In [52]:
df = df[df['start.time'] > '2010-12-31 23:59:59']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63960 entries, 52183 to 116142
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start.time        63960 non-null  datetime64[ns]
 1   peak              63960 non-null  datetime64[ns]
 2   end               63960 non-null  datetime64[ns]
 3   duration.s        63960 non-null  int64         
 4   peak.c/s          63960 non-null  int64         
 5   total.counts      63960 non-null  float64       
 6   energy.kev        63960 non-null  object        
 7   x.pos.asec        63960 non-null  int64         
 8   y.pos.asec        63960 non-null  int64         
 9   radial            63960 non-null  int64         
 10  active.region.ar  63960 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(6), object(1)
memory usage: 5.9+ MB


In [53]:
# Split the column and convert to eV
df[['energy_low_ev', 'energy_high_ev']] = df['energy.kev'].str.split('-', expand=True).astype(float) * 1000

# Convert to integer
df['energy_low_ev'] = df['energy_low_ev'].astype(int)
df['energy_high_ev'] = df['energy_high_ev'].astype(int)
df = df.drop('energy.kev', axis=1)

In [54]:
df.head()

,start.time,peak,end,duration.s,peak.c/s,total.counts,x.pos.asec,y.pos.asec,radial,active.region.ar,energy_low_ev,energy_high_ev
52183,2011-01-01 00:48:52,2011-01-01 00:51:22,2011-01-01 00:55:44,412,68,94032.0,-786,515,940,1140,6000,12000
52184,2011-01-01 02:58:40,2011-01-01 02:59:22,2011-01-01 03:00:48,128,36,20376.0,902,244,935,1138,6000,12000
52185,2011-01-01 05:41:04,2011-01-01 05:41:50,2011-01-01 05:44:20,196,28,25776.0,904,238,935,1138,6000,12000
52186,2011-01-01 05:50:28,2011-01-01 05:56:06,2011-01-01 06:01:44,676,34,94416.0,-739,565,930,1140,6000,12000
52187,2011-01-01 12:01:04,2011-01-01 12:01:38,2011-01-01 12:02:32,88,28,9192.0,-619,-179,645,0,6000,12000


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63960 entries, 52183 to 116142
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start.time        63960 non-null  datetime64[ns]
 1   peak              63960 non-null  datetime64[ns]
 2   end               63960 non-null  datetime64[ns]
 3   duration.s        63960 non-null  int64         
 4   peak.c/s          63960 non-null  int64         
 5   total.counts      63960 non-null  float64       
 6   x.pos.asec        63960 non-null  int64         
 7   y.pos.asec        63960 non-null  int64         
 8   radial            63960 non-null  int64         
 9   active.region.ar  63960 non-null  int64         
 10  energy_low_ev     63960 non-null  int64         
 11  energy_high_ev    63960 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(8)
memory usage: 6.3 MB


In [56]:
# Split the data
X = df.drop(['start.time','peak','end',], axis=1)
y = df['energy_low_ev']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
def remove_outliers_iqr(df, columns):
    df_cleaned = df.copy()
    
    for column in columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df_cleaned[column].quantile(0.25)
        Q3 = df_cleaned[column].quantile(0.75)
        IQR = Q3 - Q1  
        
        # Define bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Remove outliers
        df_cleaned = df_cleaned[(df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)]
        
    return df_cleaned

df_cleaned = remove_outliers_iqr(df, X.columns)

In [58]:
# Scale the training and testing data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [59]:
X_train_scaled

array([[ 3.3147105 ,  3.68280906,  4.90645424, ...,  0.85695243,
        -0.23396225, -0.20213812],
       [-0.69599261, -0.22208399, -0.17276263, ..., -1.32081549,
        -0.23396225, -0.20213812],
       [-0.72390933, -0.25975177, -0.19347783, ...,  0.35041811,
        -0.23396225, -0.20213812],
       ...,
       [-0.73321491, -0.24719584, -0.18579626, ..., -0.04514014,
        -0.23396225, -0.20213812],
       [-0.67738146,  0.04159046, -0.17024385, ...,  0.32624511,
        -0.23396225, -0.20213812],
       [-0.70529818, -0.26916871, -0.19154074, ...,  1.2920665 ,
        -0.23396225, -0.20213812]])

In [60]:
knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train_scaled, y_train)
    
# Make predictions on training data (for reference)
train_predictions = knn.predict(X_train_scaled)

# Make predictions on test data (for evaluation)
test_predictions = knn.predict(X_test_scaled)

# Evaluate performance on training data
train_mse = mean_squared_error(y_train, train_predictions)
train_rmse = np.sqrt(train_mse)
print(f'Training Root Mean Squared Error: {train_rmse:.2f}')

Training Root Mean Squared Error: 264.88
